In [ ]:
!pip3 install zstandard

import zstandard as zstd
import json
import pandas as pd


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [ ]:
submission_path = '/Users/brian_wei/Desktop/data/unsupportive/submissions/you_should_know_submissions'
comment_path = '/Users/brian_wei/Desktop/data/unsupportive/comments/you_should_know_comments'

In [ ]:
submission_chunks = []
comment_chunks = []

chunk_size = 100000
for chunk in pd.read_json(submission_path, lines=True, chunksize=chunk_size):
    submission_chunks.append(chunk)

submission_df = pd.concat(submission_chunks, ignore_index=True)
for chunk in pd.read_json(comment_path, lines=True, chunksize=chunk_size):
    comment_chunks.append(chunk)

comment_df = pd.concat(comment_chunks, ignore_index=True)

In [ ]:
comment_df['submission_id'] = comment_df['parent_id'].str[3:]
columns_to_include = ['body', 'score', 'created_utc']
comment_df['dict_representation'] = comment_df.apply(
    lambda row: {col: row[col] for col in columns_to_include}, axis=1)
grouped_comments = comment_df.groupby('submission_id')['dict_representation'].apply(list).reset_index(name='all_comments')
combined_df = pd.merge(submission_df, grouped_comments, left_on='id', right_on='submission_id', how='left')
combined_df.drop(columns=['submission_id'], inplace=True)

In [ ]:
combined_df.shape

(93956, 124)

In [ ]:
columns_to_keep = ['title', 'selftext', 'created_utc', 'score', 'all_comments']
combined_df = combined_df[columns_to_keep]

In [ ]:
cleaned_df = combined_df.dropna(subset=['all_comments'])
cleaned_df = cleaned_df[cleaned_df['selftext'] != '']

In [ ]:
cleaned_df.shape

(66922, 5)

In [ ]:
path = '/Users/brian_wei/Desktop/data/unsupportive/combined/you_should_know.csv'
cleaned_df.to_csv(path, index=False)
print(f'DataFrame written to {path}')

DataFrame written to /Users/brian_wei/Desktop/data/unsupportive/combined/you_should_know.csv
